## Imports

In [89]:
from tqdm.notebook import tqdm
from pprint import pprint
from decimal import Decimal
from bson.decimal128 import Decimal128
from datetime import date

## Connection

In [15]:
# Connect to mySQL server
import mysql.connector

def connect_mysql():

    mydb = mysql.connector.connect(host='rhea.isegi.unl.pt', 
                                   user='wwi-read-only-user', 
                                   database='WWI',
                                   password='jGp2GCqrss6nfTEu5ZawhW3mksLsQYQb',
                                   port=3306
                                  )
    mycursor = mydb.cursor()
    
    return mydb, mycursor

    # Getting the table names
    #mycursor.execute('SHOW TABLES;')
    #print(f"Tables: {mycursor.fetchall()}")

    # Getting a tables column descriptios
    #mycursor.execute('DESCRIBE Purchasing_PurchaseOrderLines;')
    #print(f"Purchasing_PurchaseOrderLines schema: {mycursor.fetchall()}")
    
mydb, mycursor = connect_mysql()

In [16]:
# Connect to Mongo server

from pymongo import MongoClient

host="rhea.isegi.unl.pt"
port="27049"
user="GROUP_32"
password="bRG2XjRZhrRA9IfpmENyXxMlWQDUJdzL"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

# Connect to mongo db
db = client.#database name here

In [17]:
# Get mysql tables
mycursor.execute('SHOW TABLES;')
tables = mycursor.fetchall()

## For deleting everything and flushing the cursor

In [204]:
db.list_collection_names()

['Sales_InvoiceLines',
 'Sales_Orders',
 'Sales_CustomerTransactions',
 'Purchasing_SupplierTransactions',
 'Sales_OrderLines',
 'Sales_Customers',
 'Purchasing_PurchaseOrders',
 'Purchasing_PurchaseOrderLines',
 'Purchasing_SupplierCategories',
 'Sales_CustomerCategories',
 'Sales_Invoices',
 'Purchasing_Suppliers']

In [205]:
for i in range(len(tables)):
    
    table_name = tables[i][0]
    
    db[table_name].drop()

In [206]:
db.list_collection_names()

[]

In [10]:
mycursor.fetchall()

[]

## Get row counts of each table

In [7]:
for i in range(len(tables)):
    
    # Get MySQL table name
    table_name = tables[i][0]
    
    # Define our query
    query = ( "SELECT * " 
             "FROM " + table_name )
    
    # Execute query
    mycursor.execute( query )
    
    # Print row counts
    print(table_name)
    print(len(mycursor.fetchall()))

Purchasing_PurchaseOrderLines
8367
Purchasing_PurchaseOrders
2074
Purchasing_SupplierCategories
9
Purchasing_SupplierTransactions
2438
Purchasing_Suppliers
13
Sales_CustomerCategories
8
Sales_CustomerTransactions
52000
Sales_Customers
663
Sales_InvoiceLines
30000
Sales_Invoices
70510
Sales_OrderLines
76000
Sales_Orders
73595


## Migrate data from MySQL database to mongo database

In [207]:
# Adapted from:
# https://nicksardo.wordpress.com/2015/11/24/transferring-data-between-mysql-and-mongodb/

def migrate_table(table_index):
    
    # Get MySQL table name
    table_name = tables[table_index][0]
    
    # Print table name
    print('Migrating table', table_name)
    
    # Get names of the columns of this table
    describe = 'DESCRIBE ' + table_name + ';'
    mycursor.execute(describe)
    describe_out = mycursor.fetchall()

    cols = []
    for col_index in range(len(describe_out)):
        col = describe_out[col_index][0]
        cols.append(col)
    
    # Create mongodb collection
    collection = db[table_name]
    
    # Define our query
    query = ( "SELECT * " 
             "FROM " + table_name )
    
    # Get row count and print it
    mycursor.execute( query )
    print('Rows in this table:', str(len(mycursor.fetchall())))
    
    # Execute query again
    mycursor.execute( query )
    
    #mongo client specifically requires python dict
    cus = dict()

    #custom record id rather than mongodb default hash id                                          
    cid = 0                                                 

    #cycle through each mySQL row
    for ( row ) in tqdm(mycursor):
        cid        += 1   #increment id
        cus['_id'] = cid                                    

        #check if current row is null
        for i in range( 0, len( row ) ):
            
            if row[i] == None:
                #if the record is null, skip it                  
                continue
            
            else:
                row_title      = "".join( cols[i] )  
                
                field          = row[i]
                
                if isinstance(field, Decimal):
                    field = float(str(field))
                    
                if isinstance(field, date):
                    field = str(field)
                
                #add current record's field's title and value             
                cus[row_title] = field
                
        #we've completed processing this row, insert it into mongoldb      
        collection.insert_one( cus )

In [208]:
def migrate_table_10k_entries(table_index, ten_k_step):
    
    from_entry = (ten_k_step * 10000) + 1
    till_entry = ((ten_k_step + 1) * 10000) + 1
    print('Migrate entries larger or equal to', str(from_entry), 'and smaller than', str(till_entry))
    
    # Get MySQL table name
    table_name = tables[table_index][0]
    
    # Print table name
    print('Migrating table', table_name)
    
    # Get names of the columns of this table
    describe = 'DESCRIBE ' + table_name + ';'
    mycursor.execute(describe)
    describe_out = mycursor.fetchall()

    cols = []
    for col_index in range(len(describe_out)):
        col = describe_out[col_index][0]
        cols.append(col)
    
    # Create mongodb collection
    collection = db[table_name]
    
    collection.delete_many({'_id' : { "$gte": from_entry}})
    
    # Define our query
    query = ( "SELECT * " 
             "FROM " + table_name )
    
    # Get row count and print it
    mycursor.execute( query )
    print('Rows in this table:', str(len(mycursor.fetchall())))
    
    # Execute query again
    mycursor.execute( query )
    
    #mongo client specifically requires python dict
    cus = dict()

    #custom record id rather than mongodb default hash id                                          
    cid = 0                                                 

    #cycle through each mySQL row
    for ( row ) in tqdm(mycursor):
        cid        += 1   #increment id
        cus['_id'] = cid   
        
        if cid >= from_entry and cid < till_entry:
            
            #check if current row is null
            for i in range( 0, len( row ) ):

                if row[i] == None:
                    #if the record is null, skip it                  
                    continue

                else:
                    row_title      = "".join( cols[i] )  

                    field          = row[i]

                    if isinstance(field, Decimal):
                        field = float(str(field))

                    if isinstance(field, date):
                        field = str(field)

                    #add current record's field's title and value             
                    cus[row_title] = field

            #we've completed processing this row, insert it into mongoldb      
            collection.insert_one( cus )

In [262]:
i = 11
i

11

In [263]:
tables[i][0]

'Sales_Orders'

In [211]:
db[tables[i][0]].drop()
migrate_table(i)

Migrating table Purchasing_PurchaseOrderLines
Rows in this table: 8367


0it [00:00, ?it/s]

In [212]:
db[tables[i][0]].count_documents({})

8367

In [214]:
db[tables[i][0]].drop()
migrate_table(i)

Migrating table Purchasing_PurchaseOrders
Rows in this table: 2074


0it [00:00, ?it/s]

In [215]:
db[tables[i][0]].count_documents({})

2074

In [217]:
db[tables[i][0]].drop()
migrate_table(i)

Migrating table Purchasing_SupplierCategories
Rows in this table: 9


0it [00:00, ?it/s]

In [218]:
db[tables[i][0]].count_documents({})

9

In [220]:
db[tables[i][0]].drop()
migrate_table(i)

Migrating table Purchasing_SupplierTransactions
Rows in this table: 2438


0it [00:00, ?it/s]

In [221]:
db[tables[i][0]].count_documents({})

2438

In [223]:
db[tables[i][0]].drop()
migrate_table(i)

Migrating table Purchasing_Suppliers
Rows in this table: 13


0it [00:00, ?it/s]

In [224]:
db[tables[i][0]].count_documents({})

13

In [226]:
db[tables[i][0]].drop()
migrate_table(i)

Migrating table Sales_CustomerCategories
Rows in this table: 8


0it [00:00, ?it/s]

In [227]:
db[tables[i][0]].count_documents({})

8

In [229]:
db[tables[i][0]].drop()
migrate_table(i)

Migrating table Sales_CustomerTransactions
Rows in this table: 52000


0it [00:00, ?it/s]

In [230]:
db[tables[i][0]].count_documents({})

52000

In [232]:
db[tables[i][0]].drop()
migrate_table(i)

Migrating table Sales_Customers
Rows in this table: 663


0it [00:00, ?it/s]

In [233]:
db[tables[i][0]].count_documents({})

663

In [235]:
db[tables[i][0]].drop()
migrate_table(i)

Migrating table Sales_InvoiceLines
Rows in this table: 30000


0it [00:00, ?it/s]

In [236]:
db[tables[i][0]].count_documents({})

30000

In [238]:
db[tables[i][0]].drop()
migrate_table(i)

Migrating table Sales_Invoices
Rows in this table: 70510


0it [00:00, ?it/s]

In [239]:
db[tables[i][0]].count_documents({})

70510

In [241]:
db[tables[i][0]].drop()
migrate_table(i)

Migrating table Sales_OrderLines
Rows in this table: 76000


0it [00:00, ?it/s]

OperationalError: 2013 (HY000): Lost connection to MySQL server during query

In [242]:
db[tables[i][0]].count_documents({})

32187

In [245]:
migrate_table_10k_entries(i, 3)

Migrate entries larger or equal to 30001 and smaller than 40001
Migrating table Sales_OrderLines
Rows in this table: 76000


0it [00:00, ?it/s]

OperationalError: 2013 (HY000): Lost connection to MySQL server during query

In [249]:
db[tables[i][0]].count_documents({})

40000

In [250]:
migrate_table_10k_entries(i, 4)

Migrate entries larger or equal to 40001 and smaller than 50001
Migrating table Sales_OrderLines
Rows in this table: 76000


0it [00:00, ?it/s]

OperationalError: 2013 (HY000): Lost connection to MySQL server during query

In [254]:
db[tables[i][0]].count_documents({})

50000

In [255]:
migrate_table_10k_entries(i, 5)

Migrate entries larger or equal to 50001 and smaller than 60001
Migrating table Sales_OrderLines
Rows in this table: 76000


0it [00:00, ?it/s]

In [256]:
db[tables[i][0]].count_documents({})

60000

In [257]:
migrate_table_10k_entries(i, 6)

Migrate entries larger or equal to 60001 and smaller than 70001
Migrating table Sales_OrderLines
Rows in this table: 76000


0it [00:00, ?it/s]

In [258]:
db[tables[i][0]].count_documents({})

70000

In [259]:
migrate_table_10k_entries(i, 7)

Migrate entries larger or equal to 70001 and smaller than 80001
Migrating table Sales_OrderLines
Rows in this table: 76000


0it [00:00, ?it/s]

In [260]:
db[tables[i][0]].count_documents({})

76000

In [264]:
db[tables[i][0]].drop()
migrate_table(i)

Migrating table Sales_Orders
Rows in this table: 73595


0it [00:00, ?it/s]

OperationalError: 2013 (HY000): Lost connection to MySQL server during query

In [268]:
db[tables[i][0]].count_documents({})

40017

In [269]:
migrate_table_10k_entries(i, 4)

Migrate entries larger or equal to 40001 and smaller than 50001
Migrating table Sales_Orders
Rows in this table: 73595


0it [00:00, ?it/s]

OperationalError: 2013 (HY000): Lost connection to MySQL server during query

In [273]:
db[tables[i][0]].count_documents({})

50000

In [274]:
migrate_table_10k_entries(i, 5)

Migrate entries larger or equal to 50001 and smaller than 60001
Migrating table Sales_Orders
Rows in this table: 73595


0it [00:00, ?it/s]

In [275]:
db[tables[i][0]].count_documents({})

60000

In [276]:
migrate_table_10k_entries(i, 6)

Migrate entries larger or equal to 60001 and smaller than 70001
Migrating table Sales_Orders
Rows in this table: 73595


0it [00:00, ?it/s]

In [277]:
db[tables[i][0]].count_documents({})

70000

In [278]:
migrate_table_10k_entries(i, 7)

Migrate entries larger or equal to 70001 and smaller than 80001
Migrating table Sales_Orders
Rows in this table: 73595


0it [00:00, ?it/s]

In [279]:
db[tables[i][0]].count_documents({})

73595

In [271]:
mycursor.fetchall()

[]

In [272]:
mydb, mycursor = connect_mysql()

## Check newly created mongo database

In [280]:
db.list_collection_names()

['Sales_CustomerTransactions',
 'Sales_Invoices',
 'Warehouse_StockItemTransactions',
 'Sales_CustomerCategories',
 'Purchasing_SupplierTransactions',
 'Purchasing_PurchaseOrders',
 'Warehouse_StockItemStockGroups',
 'Warehouse_Colors',
 'Sales_OrderLines',
 'Purchasing_Suppliers',
 'Warehouse_StockGroups',
 'Warehouse_StockItems',
 'Sales_InvoiceLines',
 'Sales_Customers',
 'Warehouse_PackageTypes',
 'Purchasing_PurchaseOrderLines',
 'Purchasing_SupplierCategories',
 'Sales_Orders']

In [281]:
len(db.list_collection_names())

18

In [282]:
for collection in db.list_collection_names():
    print(collection)
    pprint(db[collection].find_one())
    print()

Sales_CustomerTransactions
{'AmountExcludingTax': 2300.0,
 'CustomerID': 832,
 'CustomerTransactionID': 2,
 'FinalizationDate': '2013-01-02',
 'InvoiceID': 1,
 'IsFinalized': 1,
 'OutstandingBalance': 0.0,
 'TaxAmount': 345.0,
 'TransactionAmount': 2645.0,
 'TransactionDate': '2013-01-01',
 'TransactionTypeID': 1,
 '_id': 1}

Sales_Invoices
{'AccountsPersonID': 3032,
 'BillToCustomerID': 832,
 'ConfirmedDeliveryTime': '2013-01-02 07:05:00',
 'ConfirmedReceivedBy': 'Aakriti Byrraju',
 'ContactPersonID': 3032,
 'CustomerID': 832,
 'CustomerPurchaseOrderNumber': '12126',
 'DeliveryInstructions': 'Suite 24, 1345 Jun Avenue',
 'DeliveryMethodID': 3,
 'DeliveryRun': '',
 'InvoiceDate': '2013-01-01',
 'InvoiceID': 1,
 'IsCreditNote': 0,
 'OrderID': 1,
 'PackedByPersonID': 14,
 'ReturnedDeliveryData': '{"Events": [{ "Event":"Ready for '
                         'collection","EventTime":"2013-01-01T12:00:00","ConNote":"EAN-125-1051"},{ '
                         '"Event":"DeliveryAttempt","Even

In [283]:
# Compare sql table sizes to mongo collection sizes
for i in range(len(tables)):
    
    # Get MySQL table name
    table_name = tables[i][0]
    
    # Define our query
    query = ( "SELECT * " 
             "FROM " + table_name )
    
    # Execute query
    mycursor.execute( query )
    
    # Get row count for mongo db
    mongo_count = db[table_name].count_documents({})
    
    # Print row counts
    print(table_name)
    print('SQL db count:')
    print(len(mycursor.fetchall()))
    print('Mongo db count:')
    print(mongo_count)

Purchasing_PurchaseOrderLines
SQL db count:
8367
Mongo db count:
8367
Purchasing_PurchaseOrders
SQL db count:
2074
Mongo db count:
2074
Purchasing_SupplierCategories
SQL db count:
9
Mongo db count:
9
Purchasing_SupplierTransactions
SQL db count:
2438
Mongo db count:
2438
Purchasing_Suppliers
SQL db count:
13
Mongo db count:
13
Sales_CustomerCategories
SQL db count:
8
Mongo db count:
8
Sales_CustomerTransactions
SQL db count:
52000
Mongo db count:
52000
Sales_Customers
SQL db count:
663
Mongo db count:
663
Sales_InvoiceLines
SQL db count:
30000
Mongo db count:
30000
Sales_Invoices
SQL db count:
70510
Mongo db count:
70510
Sales_OrderLines
SQL db count:
76000
Mongo db count:
76000
Sales_Orders
SQL db count:
73595
Mongo db count:
73595
